In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from adabmDCA.utils import get_device, get_dtype
from adabmDCA.sampling import get_sampler

import sys
sys.path.append('..')
import selex_dca

import pickle

In [4]:
max_epochs = 300
pseudocount = 0.0

experiment_id = 'Dop8V030'
round_ids = ["ARN", "R01", "R02N"]

device = get_device("")
dtype = get_dtype("float32")

pc_str = format(pseudocount, '.8f')

filename = experiment_id + "freq_pseudocount" + pc_str + ".pkl"
filepath = "saved/freq_pseudocount/" + filename

with open(filepath, 'rb') as f:
    fi, fij, total_reads = pickle.load(f)

n_rounds, L, q = fi.size()
sampler_alg = "gibbs"
sampler = torch.jit.script(get_sampler(sampler_alg))

Running on CPU


In [5]:
nchains = 10**4

params = selex_dca.init_parameters(fi=fi) # initialize with frequences at last round
chains = selex_dca.init_chains(num_rounds=n_rounds, num_chains=nchains, L=L, q=q, device=device, fi=fi)

history = selex_dca.init_history()
log_weights = torch.zeros(n_rounds, nchains, device=device, dtype=dtype)

nsweeps = 10
lr = 0.01

ch, par, history = selex_dca.train(
    sampler=sampler,
    chains=chains,
    fi=fi,
    fij=fij,
    total_reads=total_reads,
    params=params,
    nsweeps=nsweeps,
    lr=lr,
    max_epochs=max_epochs,
    target_pearson=0.99,
    history=history,
    log_weights=log_weights,
    progress_bar=True
    )

 0.00%[                                             ] Epoch: 0/300 [00:00]

KeyboardInterrupt: 